In [1]:
import pandas as pd
import numpy as np


# download config_bdd30-100.pkl here 
# https://filesender.renater.fr/?s=download&token=556ae671-4546-493e-f96f-3cd3c1919724#

# TODO: excellent "test" set since 15K and 30K have not been subject to specialization
# pd.read_csv('config_bdd15000-20000.csv') 
# pd.read_csv('config_bdd20000-30000.csv') 
# 
#rawtuxdata = pd.read_csv('../tuxml-datasets/config_bdd100000-1265000.csv') 
#rawtuxdata = pd.read_csv('../tuxml-datasets/config_bdd60000-90000.csv') 
rawtuxdata = pd.read_pickle('config_bdd30-100.pkl')
#rawtuxdata = pd.read_csv('dataset_after_encoding.csv') # TODO: Hugo?
basic_head = ["cid", "time", "date"] # "compile"
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

print("some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course")
# we only keep X86_64 configurations
#rawtuxdata = rawtuxdata[rawtuxdata['X86_64'] == 'y'] ### TODO: I've impression it's not the most effective way (wrt memory) to filter 
if 'X86_64' in rawtuxdata.columns:
    print(rawtuxdata['X86_64'].describe())
    rawtuxdata.query("X86_64 == 'y'", inplace=True) # only keep them


some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course
count     69459
unique        2
top           y
freq      69423
Name: X86_64, dtype: object


In [2]:
rawtuxdata["time"][:10]

0    428.014
1    241.167
2    198.649
3    214.034
4    348.474
5    658.585
6    248.957
7    545.106
8    166.481
9    244.012
Name: time, dtype: float64

In [16]:
list(map(lambda x: x / 60, list((np.sum(rawtuxdata["time"]), np.mean(rawtuxdata["time"]), np.min(rawtuxdata["time"]), np.max(rawtuxdata["time"])))))

[625822.15047, 9.014622682252279, 0.05381416666666667, 225.68333333333334]

In [ ]:
### TODO: ensure that the same configuration leads to the same compilation status (normally it's the case)
# remove entries with same configurations
print(str(len(rawtuxdata)) + " before the removal of some entries (those with same configurations)")
# tuxdata.drop_duplicates(subset=tuxdata.columns.difference(['vmlinux']), inplace=True)
rawtuxdata.drop_duplicates(subset=rawtuxdata.columns.difference(size_methods).difference(basic_head), inplace=True)
print(str(len(rawtuxdata)) + " after the removal of some entries (those with same configurations)")

In [ ]:
suspicious_cids = [70715, 70716, 74459, 74460, 74461, 74463, 74464] # see cluster analysis
for cid in suspicious_cids:
    print("cid", cid, "exists?", len(rawtuxdata.query("cid == " + str(cid)).index))


In [ ]:
cid_to_remove=74459 # compilation time is very low (2 sec)
cid_to_remove2=30698 # size is 0! 

rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove]
rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove2]

In [ ]:
# assert (len(rawtuxdata) == 20001)
len(rawtuxdata)
rawtuxdata

In [ ]:
len(rawtuxdata.query("vmlinux == -1").index)

In [ ]:
def nbyes(row):
    return sum(row == "y")                    
rawtuxdata['nbyes'] = rawtuxdata.apply(nbyes, axis=1)

In [ ]:
rawtuxdata.sort_values(by="nbyes")[:10]

In [ ]:
# tinykernel
rawtuxdata.query("vmlinux == 7317008") #.iloc[0]

In [ ]:
rawtuxdata.query("GENERIC_ALLOCATOR == 'y'") # & DRM_VBOXVIDEO == 'y'")
#rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")
#rawtuxdata.query("X86_64 == 'y'")

In [ ]:
rawtuxdata.query("vmlinux != -1").sort_values(by="vmlinux")[:20]

In [ ]:
compilation_failures = len(rawtuxdata.query("vmlinux == -1").index)
compilation_successes = len(rawtuxdata.query("vmlinux != -1").index)
n_compilations = len(rawtuxdata.index)
assert(compilation_successes + compilation_failures == n_compilations)
compilation_failure_percentage = (compilation_failures / n_compilations) * 100
print("compilation failures:", compilation_failures, "out of", n_compilations, "(", compilation_failure_percentage, "%)")

In [ ]:
# BUGS EXPLORATION
def bug_exploration():
    #rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]
    #rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'm'")[['cid', 'vmlinux']]
    return rawtuxdata.query("DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]

#bug_exploration()


# y or n values
n_AIC7XXX_alone = len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").index)
n_AIC79XX_alone = len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").index)
# masking effects!
n_AIC_both = len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'").index)
n_AIC_or = len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y' | AIC79XX_BUILD_FIRMWARE == 'y'").index)
print("AIC7XXX_BUILD_FIRMWARE (without 79XX)", n_AIC7XXX_alone - n_AIC_both, "AIC79XX_BUILD_FIRMWARE (without X7XX)", n_AIC79XX_alone - n_AIC_both)
print("AIC7XXX_BUILD_FIRMWARE or AIC79XX_BUILD_FIRMWARE",  n_AIC_or)
print("Without the 2 major configuration bugs, caused by 2 individual options, it remains", len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index))
# masking effects!
# option WANXL_BUILD_FIRMWARE responsible of many bugs, except... 1
n_wanxl = len(rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").index)
print("WANXL_BUILD_FIRMWARE", n_wanxl)
# same as previously, but we only keep compilation failures... and surprise:
n_wanxl_failure = len(rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1)").index)
print("There is apparently one subtle case with WANXL_BUILD_FIRMWARE=y causing **NO** configuration bug (with VIDEO_RCAR_VIN)", n_wanxl_failure) # len(rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & vmlinux == -1").index)

assert ((compilation_failures - (n_AIC_or + n_wanxl_failure)) == len(rawtuxdata.query("WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index))
print("still tracking", (compilation_failures - (n_AIC_or + n_wanxl_failure)), "configuration bugs")
print("let's now investigate GENERIC_ALLOCATOR")
len(rawtuxdata.query("GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index), len(rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index)
#len(rawtuxdata.query("WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1").index)
#len(rawtuxdata.query("vmlinux == -1").index)

#n_AIC_or + n_wanxl

In [ ]:
rawtuxdata.sort_values(by='cid')

In [ ]:
# Juliana 
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values
#len(rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").index)

# Djamel 
rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").sample(n=10)['cid'].values
len(rawtuxdata.query("AIC79XX_BUILD_FIRMWARE == 'y'").index)

# Arnaud 
#rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')").sample(n=10)['cid'].values
#rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE == 'y' | AIC79XX_BUILD_FIRMWARE == 'y')")['cid'].values
# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].values

# rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")['cid'].values

# Mathieu 
# GENERIC_ALLOCATOR == 'y' + DRM_VBOXVIDEO
# interestingly: 'm' (module) does not lead to a bug! < 1.5
# ['vmlinux', 'cid']
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].values
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")['cid'].values
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")['cid'].values

In [ ]:
# masking effect
# Mathieu
## is there a masking effect between?
#rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC79XX_BUILD_FIRMWARE == 'y'")
## is there a masking effect between?
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & AIC7XXX_BUILD_FIRMWARE == 'y'")
## is there a masking effect between?
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE == 'y'")
# rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")

In [ ]:
rawtuxdata.info(memory_usage='deep')

In [ ]:
# https://github.com/TuxML/ProjetIrma/issues/137
# We should check whether in the database we have configurations in which we have options values:
# CONFIG_WANXL_BUILD_FIRMWARE=y and CONFIG_PREVENT_FIRMWARE_BUILD=y
# Hypothesis: The decision tree showed that when CONFIG_WANXL_BUILD_FIRMWARE is activated, 
# there are 335 failures out of 337... 
# As discussed, the "two" exceptional cases causing failures were quite intriguing... And maybe you get the explanation! Strangely, and according to the decision tree, the activation of option "CONFIG_VIDEO_RCAR_VIN" avoids compilation failures. I suspect that incidentally it is a wrong explanation of non-compilation failures... and that in fact it is due to the presence of CONFIG_PREVENT_FIRMWARE_BUILD. I will verify this hypothesis!

#wanxl_configurations = rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y' & (AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y')")
#wanxl_configurations2 = rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'")

# wanxl_configurations.query("vmlinux != -1") # not necessary a failure? 
#wanxl_configurations2.query("vmlinux != -1")

#rawtuxdata.query("PREVENT_FIRMWARE_BUILD == 'y' & WANXL_BUILD_FIRMWARE == 'y'") #.query("vmlinux != -1")

rawtuxdata.query("WANXL_BUILD_FIRMWARE == 'y'") #  & vmlinux != -1")
#rawtuxdata.query("VIDEO_RCAR_VIN == 'm'")



In [ ]:
### basic stats about options and unique values 
## could be extended/improved 

tri_state_values = ['y', 'n', 'm']

ftuniques = []
freq_ymn_features = []
non_tristate_options = []

for col in rawtuxdata:
    ft = rawtuxdata[col]    
    # eg always "y"
    if len(ft.unique()) == 1:
        ftuniques.append(col)
    # only tri-state values (y, n, m) (possible TODO: handle numerical/string options)    
    elif all(x in tri_state_values for x in ft.unique()):     #len(ft.unique()) == 3: 
        freq = ft.value_counts(normalize=True)
        freqy = 0
        freqn = 0
        freqm = 0
        if ('y' in freq.index):
            freqy = freq['y']
        if ('n' in freq.index):
            freqn = freq['n']
        if ('m' in freq.index):
            freqm = freq['m']
        freq_ymn_features.append((col, freqy, freqm, freqn))
    else:
        if not (col in size_methods): 
            non_tristate_options.append(col)
        

### TODO: we want to keep all quantitative values!
# non_tristate_options.remove('LZO') # ('vmlinux')

# we want to keep measurements (that are not tristate ;)) 
# non_tristate_options = list(set(non_tristate_options) - set(size_methods))

#### print options with unique values
# options with only one value eg always "y"
#i = 0
#for ft in ftuniques:
#    print(ft + " (" + str(i) + ")")
#    i = i + 1

print("Original size (#configs/#options) of the dataset " + str(rawtuxdata.shape))
print ("Number of options with only one value (eg always y): " + str(pd.DataFrame(ftuniques).shape))

# maybe we can drop options with only one unique value (no interest for machine learning)
# TODO: maybe we can rely on more traditional feature reduction techniques
# TODO: need to think about *when* to apply the removal 

#rawtuxdata.drop(columns=ftuniques,inplace=True) 

## non_tristate_options include basic stuff like date, time, cid but also string/numerical options
print ("Non tri-state value options (eg string or integer or hybrid values): " 
       + str(pd.DataFrame(non_tristate_options).shape) + " ") 
#      + str(pd.DataFrame(non_tristate_options)))


print ("Predictor variables: " + str(rawtuxdata.drop(columns=non_tristate_options).columns.size))
# frequency of y, m, and n values 
#plt.figure()
#pd.DataFrame(freq_ymn_features, columns=["feature", "freqy", "freqm", "freqn"]).plot(kind='hist', alpha=0.8) #plot()
#plt.show()


rawtuxdata.info(memory_usage='deep')

In [ ]:
from sklearn.preprocessing import *
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score
from sklearn import tree
import graphviz 


LEARN_COMPILATION_SUCCESS = True # costly in time and space 
compilation_status_column_name = 'compile_success'

def encode_data_compilation(rawtuxdata):
    lae = LabelEncoder()
    # we save quantitative values we want (here vmlinux, TODO: generalize)
    # the key idea is that the labelling encoder should not be applied to this kind of values (only to predictor variables!)
    # vml = rawtuxdata['LZO'] # rawtuxdata['vmlinux'] 
    o_sizes = rawtuxdata[size_methods]

    # we may remove non tri state options, but TODO there are perhaps some interesting options (numerical or string) here
    #tuxdata = rawtuxdata.drop(columns=non_tristate_options).drop(columns=['vmlinux']).apply(le.fit_transform)
    tuxdata_for_compilation = rawtuxdata.drop(columns=non_tristate_options).drop(columns=size_methods).apply(lae.fit_transform)

    #tuxdata['vmlinux'] = vml 
    tuxdata_for_compilation[size_methods] = o_sizes
    # we can ue vmlinux since it has been restored thanks to previous line
    tuxdata_for_compilation[compilation_status_column_name] = tuxdata_for_compilation['vmlinux'] != -1
    return tuxdata_for_compilation

def learn_compilation_success(tuxdata_for_compilation):
    TESTING_SIZE=0.001 # 0.001 
    X_train, X_test, y_train, y_test = train_test_split(tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name), tuxdata_for_compilation[compilation_status_column_name], test_size=TESTING_SIZE, random_state=0)  
    clf = tree.DecisionTreeClassifier() #GradientBoostingClassifier(n_estimators=100) #RandomForestRegressor(n_estimators=100) #   #GradientBoostingRegressor(n_estimators=100)  
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]    

    TOP_FT_IMPORTANCE=20
    print("Feature ranking: " + "top (" + str(TOP_FT_IMPORTANCE) + ")")
    for f in range(TOP_FT_IMPORTANCE): # len(indices)
        print("%d. feature %s %d (%f)" % (f + 1, tuxdata_for_compilation.columns[indices[f]], indices[f], importances[indices[f]]))
   
    
    dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name).columns,  
                         filled=True, rounded=True,
                         special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph.render("TUXML_compilation_failures")
    
    acc = accuracy_score (y_test, y_pred)
    prec = precision_score (y_test, y_pred)
    reca = recall_score (y_test, y_pred)
    f1 = f1_score (y_test, y_pred)
    balance_acc = balanced_accuracy_score (y_test, y_pred)
    print("Accuracy score:", acc)
    print("Precision score:", prec)
    print("Recall score:", reca)
    print("F1 score:", f1)
    print("Balance accuracy score:", balance_acc)

if (LEARN_COMPILATION_SUCCESS):
    tuxdata_for_compilation = encode_data_compilation(rawtuxdata)
    # tuxdata_for_compilation [compilation_status_column_name].describe() # TODO?
    learn_compilation_success(tuxdata_for_compilation)
    del tuxdata_for_compilation

In [ ]:
### CRC32 == m?
# VIDEO_OV5640 == 'y'? 
# rawtuxdata[['VIDEO_V4L2', 'MEDIA_TUNER']]
# & MEDIA_TUNER != 'n' 
# VIDEO_V4L2 != 'n' & 
# V4L2_FWNODE
rawtuxdata.query("VIDEO_MUX == 'y' & VIDEO_OV5640 == 'm' & CRC32 != 'n' & UBSAN_SANITIZE_ALL != 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1")[['VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # ['cid'].values
#rawtuxdata.query("VIDEO_V4L2 == 'm'")[['VIDEO_V4L2', 'MEDIA_TUNER', 'vmlinux']]

In [ ]:
print("specific analysis for dataset 100K-126K")
# [['VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # 
rawtuxdata.query("REGMAP_I2C == 'm' & VIDEO_MUX == 'y' & VIDEO_XILINX_VTC == 'm' & GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE == 'n'")[['cid', 'VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # ['cid'].values
#

In [ ]:
print("specific analysis for dataset 100K-126K")
# [['VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux']] # 
rawtuxdata.query("VIDEO_MUX != 'y' & VIDEO_XILINX_VTC == 'm' & GENERIC_ALLOCATOR == 'y' & WANXL_BUILD_FIRMWARE == 'n' & vmlinux == -1")[['cid', 'vmlinux', 'SND_ATMEL_SOC_CLASSD', 'SND_ATMEL_SOC_DMA', 'SND_ATMEL_SOC_SSC_DMA', 'SND_SOC_GENERIC_DMAENGINE_PCM']]
#['cid'].values
# [['cid', 'VIDEO_V4L2', 'MEDIA_TUNER', 'V4L2_FWNODE', 'vmlinux', 'UEVENT_HELPER_PATH']] # ['cid'].values
#

In [ ]:
print("specific analysis for dataset 100K-126K")
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO == 'y'")[['cid', 'vmlinux']]['cid'].values
#

In [ ]:
print("specific analysis for dataset 100K-126K")
rawtuxdata.query("BACKLIGHT_SAHARA == 'm' & RT_MUTEXES == 'n' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y'")[['cid', 'vmlinux', 'USB_COMMON', 'USB_GADGET', 'USB', 'USB_ULPI_BUS']]

In [ ]:
print("specific analysis for dataset 100K-126K")
# PINCTRL_UNIPHIER_LD4 == 'y' & 
rawtuxdata.query("VIDEO_EXYNOS4_FIMC_IS == 'm' & VIDEO_XILINX_VTC == 'n' & WANXL_BUILD_FIRMWARE != 'y' & GENERIC_ALLOCATOR == 'y'")[['cid', 'vmlinux', 'VIDEO_MUX', 'VIDEO_V4L2', 'PINCTRL_UNIPHIER_LD4']]
# ['cid'].values
#[['cid', 'vmlinux']]

In [ ]:
print("specific analysis for dataset 100K-126K")
# PINCTRL_UNIPHIER_LD4 == 'y' & 
rawtuxdata.query("VIDEO_EXYNOS4_FIMC_IS != 'y' & VIDEO_XILINX_VTC != 'y' & WANXL_BUILD_FIRMWARE != 'y' & GENERIC_ALLOCATOR != 'n' & vmlinux == -1")['cid'].values
# [['cid', 'vmlinux', 'VIDEO_MUX', 'VIDEO_V4L2']]
#

In [ ]:
rawtuxdata.query("cid == 109530")[['ACPI', 'BACKLIGHT_CLASS_DEVICE']] 

In [ ]:
rawtuxdata.query("cid == 115367")[['cid', 'vmlinux', 'DRM_VBOXVIDEO', 'DRM', 'PCI', 'DRM_KMS_HELPER']]


In [ ]:
rawtuxdata.query("cid == 100734 | cid == 101561 | cid == 126652 | cid == 113286")[['cid', 'vmlinux', 'ACPI', 'BACKLIGHT_CLASS_DEVICE']]

In [ ]:
rawtuxdata.query(" | ".join(["cid == " + str(e) for e in [100390, 101698, 106362, 106779, 108827, 113731, 119021, 119060, 122676, 124965]]))[['cid', 'vmlinux', 'MEDIA_TUNER', 'VIDEO_MUX', 'VIDEO_V4L2']]

In [ ]:
rawtuxdata.query("cid == 121405")[['JOLIET', 'NLS_DEFAULT', 'NLS']]

In [ ]:
# 
# 
# IPV6 == 'n'
# rawtuxdata.query("(UBSAN_ALIGNMENT != 'n' | UBSAN_NULL != 'n') & IPV6 != 'm' & FORTIFY_SOURCE == 'y' & IPV6 != 'y' & INFINIBAND_ADDR_TRANS == 'y' & UBSAN_SANITIZE_ALL == 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['IPV6', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'vmlinux']]  # ['cid'].values 
rawtuxdata.query("IPV6 == 'n' & (UBSAN_ALIGNMENT != 'n' | UBSAN_NULL != 'n') & FORTIFY_SOURCE == 'y' & INFINIBAND_ADDR_TRANS == 'y' & UBSAN_SANITIZE_ALL == 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['IPV6', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'vmlinux']]  # ['cid'].values 


In [ ]:
errors = pd.read_csv('errors.csv')
clusters = []

#rawtuxdata.query("vmlinux == -1", inplace=True) 
#print (len(rawtuxdata))
#rawtuxdata['cluster'] = 'null'
#num = len(rawtuxdata.columns) - 1 
# query("vmlinux == -1").
i = 0
for row in rawtuxdata.iterrows():
    cid = row[1]['cid']
    if (row[1]['vmlinux'] != -1):
        log = 'NO_FAILURE'
    else:
        cid_error = errors.query("cid == " + str(cid))
        if (len(cid_error) > 0):
            log = cid_error['error'].iloc[0]
        else:
            print("strange case (no cid?)")
    clusters.append(log)
    #rawtuxdata.iloc[i, num] = log
#    i = i + 1
#rawtuxdata[:10]
#pd.merge(rawtuxdata, errors, on='cid', how='outer')

In [ ]:
rawtuxdata['cluster_error'] = clusters

In [ ]:
rawtuxdata.query("cluster_error != 'NO_FAILURE'")[['cid', 'cluster_error', 'vmlinux']][:10]

In [ ]:
rawtuxdata.query("vmlinux == -1").shape, rawtuxdata.query("cluster_error != 'NO_FAILURE'").shape

In [ ]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")['cluster_error'].unique()

In [ ]:
rawtuxdata.query("IPV6 == 'n' & (UBSAN_ALIGNMENT != 'n' | UBSAN_NULL != 'n') & FORTIFY_SOURCE == 'y' & INFINIBAND_ADDR_TRANS == 'y' & UBSAN_SANITIZE_ALL == 'y' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")['cluster_error'].unique()
#[['IPV6', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'vmlinux']]  # ['cid'].values 


In [ ]:
#clusters = pd.read_csv('clusters.csv')
#clusters

In [ ]:
rawtuxdata.query("MCP4725 == 'y' & CRC32 != 'y' & UBSAN_SANITIZE_ALL == 'n' & GENERIC_ALLOCATOR != 'n' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'CRC32', 'MCP4725', 'DRM_I915_GVT', 'cluster_error']]

In [ ]:
rawtuxdata.query("cluster_error == 'L6'") 

In [ ]:
# ['cid'].values # 
rawtuxdata.query("SCSI_SRP_ATTRS == 'n' & INFINIBAND_SRPT == 'y' & ISDN_DIVAS_USERIDI == 'n' & MLX5_INFINIBAND != 'y' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL']]

In [ ]:
rawtuxdata.query("MLX5_INFINIBAND == 'm' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS']]

In [ ]:
rawtuxdata.query("GPIO_GRGPIO == 'm' & ISDN_DIVAS_USERIDI == 'm' & MLX5_INFINIBAND == 'n' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS']]

In [ ]:
# ['cid'].values # 
rawtuxdata.query("GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' & vmlinux == -1")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS', 'MLX5_INFINIBAND']]

In [ ]:
rawtuxdata.query("MLX5_INFINIBAND == 'y' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y'")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS', 'MLX5_INFINIBAND']]


In [ ]:
rawtuxdata.query("cid == 65670")[['ACPI', 'BACKLIGHT_CLASS_DEVICE', 'DRM_PANEL_SAMSUNG_S6E3HA2', 'DRM_I915']]

In [ ]:
#rawtuxdata.query("vmlinux == -1 & XEN_PCIDEV_BACKEND == 'n' & WIL6210_TRACING == 'n' & INFINIBAND_SRPT == 'n' & ISDN_DIVAS_USERIDI == 'n' & MLX5_INFINIBAND != 'y' & GENERIC_ALLOCATOR == 'n' & DRM_VBOXVIDEO != 'y' & WANXL_BUILD_FIRMWARE != 'y' & AIC7XXX_BUILD_FIRMWARE != 'y' & AIC79XX_BUILD_FIRMWARE != 'y' ")[['cid', 'vmlinux', 'IPV6', 'UBSAN_SANITIZE_ALL', 'INFINIBAND_ADDR_TRANS', 'FORTIFY_SOURCE', 'UBSAN_ALIGNMENT', 'UBSAN_NULL', 'NET_FOU_IP_TUNNELS', 'MLX5_INFINIBAND', 'SCSI_SRP_ATTRS', 'RDS']]

In [ ]:
rawtuxdata.query("cid == 44596")[['AIC7XXX_BUILD_FIRMWARE', 'AIC79XX_BUILD_FIRMWARE', 'BACKLIGHT_CLASS_DEVICE', 'I2C', 'NLS', 'WANXL_BUILD_FIRMWARE ', 'vmlinux']]